In [21]:
import pandas as pd
import yaml

df=pd.read_csv('./../artifacts/data_ingestion/raw/Train.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [22]:
schema=yaml.safe_load(open('./../configs/schema.yaml'))
schema


{'columns': ['Item_Identifier',
  'Item_Weight',
  'Item_Fat_Content',
  'Item_Visibility',
  'Item_Type',
  'Item_MRP',
  'Outlet_Identifier',
  'Outlet_Establishment_Year',
  'Outlet_Size',
  'Outlet_Location_Type',
  'Outlet_Type',
  'Item_Outlet_Sales'],
 'columns_datatype': {'Item_Identifier': 'object',
  'Item_Weight': 'float64',
  'Item_Fat_Content': 'object',
  'Item_Visibility': 'float64',
  'Item_Type': 'object',
  'Item_MRP': 'float64',
  'Outlet_Identifier': 'object',
  'Outlet_Establishment_Year': 'int64',
  'Outlet_Size': 'object',
  'Outlet_Location_Type': 'object',
  'Outlet_Type': 'object',
  'Item_Outlet_Sales': 'float64'}}

In [23]:
for col in schema['columns']:
    print(col)

Item_Identifier
Item_Weight
Item_Fat_Content
Item_Visibility
Item_Type
Item_MRP
Outlet_Identifier
Outlet_Establishment_Year
Outlet_Size
Outlet_Location_Type
Outlet_Type
Item_Outlet_Sales


In [24]:
for col in df.columns:
    print(col,type(col))
    if col not in schema["columns"]:
        print(f"[ {col} ] is extra column present in dataset")
for col in schema["columns"]:
    if col not in df.columns:
        print(f"[ {col} ] column is not present in dataset")


Item_Identifier <class 'str'>
Item_Weight <class 'str'>
Item_Fat_Content <class 'str'>
Item_Visibility <class 'str'>
Item_Type <class 'str'>
Item_MRP <class 'str'>
Outlet_Identifier <class 'str'>
Outlet_Establishment_Year <class 'str'>
Outlet_Size <class 'str'>
Outlet_Location_Type <class 'str'>
Outlet_Type <class 'str'>
Item_Outlet_Sales <class 'str'>


In [25]:
schema['columns_datatype']

{'Item_Identifier': 'object',
 'Item_Weight': 'float64',
 'Item_Fat_Content': 'object',
 'Item_Visibility': 'float64',
 'Item_Type': 'object',
 'Item_MRP': 'float64',
 'Outlet_Identifier': 'object',
 'Outlet_Establishment_Year': 'int64',
 'Outlet_Size': 'object',
 'Outlet_Location_Type': 'object',
 'Outlet_Type': 'object',
 'Item_Outlet_Sales': 'float64'}

In [26]:
for col in df.columns:
    if df[col].dtype != schema["columns_datatype"][col]:
        print(f"{col} is of type [ {df[col].dtype} ], required type [ {schema['columns'][col]} ]")
